# 1. 환경설정

- 파이토치

- 한국어를 영어로 번역하는 Seq2Seq(기본 모델, Attention 적용 모델)을 구현

- JSON 파일(train_set.json, valid_set.json) 형식

- 데이터셋 각 항목은 한국어 문장("ko")과 영어 번역문("mt")으로 구성

- 적절한 토크나이저를 선택하여 한국어, 영어 문장을 토큰화

- 필요한 경우 SOS, EOS, PAD, UNK 등의 특수 토큰을 정의

- 한국어와 영어 각각의 어휘 사전 구성

- Seq2Seq 모델: GRU 기반의 Encoder-Decoder 모델을 구현하고, Teacher Forcing 기법을 적용해 학습

- Attention 모델: Attention(Bahdanau 혹은 Luong)을 적용한 디코더를 구현

- 무작위 문장 쌍에 대해 모델의 번역 결과를 출력

- 다양한 평가 지표(예: BLEU 점수) 도입

In [ ]:
""" 디렉토리 지정 """
import sys
if 'google.colab' in sys.modules:
    from google.colab import drive; drive.mount('/content/drive')
    import drive.MyDrive.develop.config_my_path as cc
    cc.dir('projects/side/250625-250702_machine_translator')

""" 라이브러리 호출 """
!pip install gensim

import os, sys
import zipfile
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from gensim.models import Word2Vec, FastText
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors

import numpy as np
import re
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict


import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')








import json
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
from torch.utils.data import DataLoader, TensorDataset, RandomSampler
from konlpy.tag import Okt
import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('punkt_tab')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# JSON 파일 경로
train_json_file_path = "/content/drive/Shareddrives/스프린트(AI) 드라이브/트랙 Master 폴더/스프린트 미션 및 모범답안/data/translation/일상생활및구어체_한영_train_set.json"
valid_json_file_path = "/content/drive/Shareddrives/스프린트(AI) 드라이브/트랙 Master 폴더/스프린트 미션 및 모범답안/data/translation/일상생활및구어체_한영_valid_set.json"

# JSON 파일 불러오기
def load_json(file_path, max_samples=1000):
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    return data["data"][:max_samples]

# 훈련 및 검증 데이터 로드
data_train = load_json(train_json_file_path, max_samples=50000)
data_valid = load_json(valid_json_file_path, max_samples=1000)

# ko와 mt 데이터 추출
ko_sentences_train = [item["ko"] for item in data_train]
mt_sentences_train = [item["mt"] for item in data_train]
ko_sentences_valid = [item["ko"] for item in data_valid]
mt_sentences_valid = [item["mt"] for item in data_valid]

# 한국어 및 영어 토크나이저
tokenizer_ko = Okt().morphs
tokenizer_en = word_tokenize

## 문장 길이 분석
ko_lengths = [len(tokenizer_ko(sent)) for sent in ko_sentences_train]
en_lengths = [len(tokenizer_en(sent)) for sent in mt_sentences_train]
all_lengths = ko_lengths + en_lengths

# 한국어와 영어 중 가장 긴 문장의 길이 기준으로 MAX_LENGTH 설정
MAX_LENGTH = max(max(ko_lengths), max(en_lengths)) + 1  # SOS, EOS 포함 고려
print(f"Max sequence length: {MAX_LENGTH}")

# 특수 토큰 정의
SOS_token = 0
EOS_token = 1
PAD_token = 2
UNK_token = 3

class Lang:
    def __init__(self, name):
        self.name = name
        # 초기에는 PAD, SOS, EOS, UNK 토큰을 미리 등록
        self.word2index = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS", UNK_token: "<unk>"}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS", UNK_token: "<unk>"}
        self.word2count = {}
        self.n_words = 4  # PAD, SOS, EOS, UNK 포함

    def addSentence(self, sentence, tokenizer):
        for word in tokenizer(sentence):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.index2word[self.n_words] = word
            self.word2count[word] = 1
            self.n_words += 1
        else:
            self.word2count[word] += 1

# 데이터 준비
def prepareData(lang1, lang2, tokenizer1, tokenizer2):
    input_lang = Lang(lang1)
    output_lang = Lang(lang2)
    pairs = list(zip(ko_sentences_train, mt_sentences_train))
    print("Read %s sentence pairs" % len(pairs))
    for pair in pairs:
        input_lang.addSentence(pair[0], tokenizer1)
        output_lang.addSentence(pair[1], tokenizer2)
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData("ko", "en", tokenizer_ko, tokenizer_en)

# 텐서 변환 및 데이터 로더 생성
def tensorFromSentence(lang, sentence, tokenizer):
    indexes = [SOS_token]
    indexes += [lang.word2index.get(word, UNK_token) for word in tokenizer(sentence)[:MAX_LENGTH - 2]]
    indexes.append(EOS_token)
    # 길이 MAX_LENGTH에 맞춰 PAD 추가
    while len(indexes) < MAX_LENGTH:
        indexes.append(PAD_token)
    return torch.tensor(indexes[:MAX_LENGTH], dtype=torch.long, device=device)

def get_dataloader(batch_size):
    input_tensors = [tensorFromSentence(input_lang, inp, tokenizer_ko) for inp, _ in pairs]
    target_tensors = [tensorFromSentence(output_lang, tgt, tokenizer_en) for _, tgt in pairs]

    input_tensors = torch.stack(input_tensors, dim=0)  # [num_samples, MAX_LENGTH]
    target_tensors = torch.stack(target_tensors, dim=0)  # [num_samples, MAX_LENGTH]

    dataset = TensorDataset(input_tensors, target_tensors)
    train_sampler = RandomSampler(dataset)
    train_dataloader = DataLoader(dataset, sampler=train_sampler, batch_size=batch_size)

    print(f"input_tensors.shape: {input_tensors.shape}, target_tensors.shape: {target_tensors.shape}")
    return train_dataloader

train_dataloader = get_dataloader(batch_size=32)







In [ ]:
import json
import matplotlib.pyplot as plt
from collections import Counter

# JSON 로딩
with open('data/train_set.json', encoding='utf-8') as f:
    raw_data = json.load(f)['data']

# 원문 추출
ko_sentences = [item['ko'].strip().replace('>', '') for item in raw_data]
en_sentences = [item['mt'].strip().replace('>', '') for item in raw_data]


In [ ]:
ko_lens = [len(s) for s in ko_sentences]
en_lens = [len(s) for s in en_sentences]

print(f"📏 한국어 길이 - 평균: {sum(ko_lens)/len(ko_lens):.2f}, 최대: {max(ko_lens)}, 최소: {min(ko_lens)}")
print(f"📏 영어 길이 - 평균: {sum(en_lens)/len(en_lens):.2f}, 최대: {max(en_lens)}, 최소: {min(en_lens)}")


In [ ]:
plt.hist(ko_lens, bins=30, alpha=0.6, label='Korean')
plt.hist(en_lens, bins=30, alpha=0.6, label='English')
plt.xlabel('문장 길이 (문자 수)')
plt.ylabel('문장 수')
plt.title('문장 길이 분포')
plt.legend()
plt.show()


In [ ]:
plt.scatter(ko_lens, en_lens, alpha=0.4)
plt.xlabel('한국어 문장 길이')
plt.ylabel('영어 문장 길이')
plt.title('문장 길이 상관관계')
plt.grid(True)
plt.show()


In [ ]:
empty_count = sum(1 for k, e in zip(ko_sentences, en_sentences) if not k or not e)
print(f"❗ 빈 문장 쌍 수: {empty_count}")

pair_set = set(zip(ko_sentences, en_sentences))
print(f"🧼 전체 샘플: {len(ko_sentences)}, 중복 제거 후: {len(pair_set)}")


In [ ]:
domains = [item['domain'] for item in raw_data]
domain_count = Counter(domains)

plt.bar(domain_count.keys(), domain_count.values())
plt.xticks(rotation=45)
plt.title("도메인 분포")
plt.ylabel("샘플 수")
plt.show()


In [ ]:
styles = [item['style'] for item in raw_data if item['style']]
style_count = Counter(styles)

print("🎭 문체 분포:", style_count)


In [ ]:
import json
from kiwipiepy import Kiwi
import spacy

def load_and_tokenize(path):
    kiwi = Kiwi()
    en_tokenizer = spacy.load("en_core_web_sm")

    def tokenize_ko(text):
        return [token.form for token in kiwi.tokenize(text)]

    def tokenize_en(text):
        return [token.text.lower() for token in en_tokenizer(text)]

    with open(path, encoding='utf-8') as f:
        raw = json.load(f)['data']
    
    ko_raw, en_raw = [], []
    ko_tokenized, en_tokenized = [], []

    for item in raw:
        ko = item['ko'].strip().replace('>', '')
        en = item['mt'].strip().replace('>', '')
        ko_raw.append(ko)
        en_raw.append(en)
        ko_tokenized.append(tokenize_ko(ko))
        en_tokenized.append(tokenize_en(en))
    
    return ko_raw, en_raw, ko_tokenized, en_tokenized, raw


ko_raw, en_raw, ko_tok, en_tok, raw_json = load_and_tokenize("data/train_set.json")

print("✅ 한국어 원문:", ko_raw[0])
print("✅ 한국어 토큰:", ko_tok[0])
print("✅ 영어 원문:", en_raw[0])
print("✅ 영어 토큰:", en_tok[0])
